<a href="https://colab.research.google.com/github/klemenp950/ElektronskaTajnica/blob/main/appointment_transcript_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Appointment Phone Call Transcript Generator

This notebook generates synthetic transcriptions of phone conversations between patients calling to schedule medical appointments and doctor's office assistants.

**Conversation Structure:**
- **spk_0**: Doctor's office assistant
- **spk_1**: Patient calling to schedule appointment

**Typical Flow:**
1. Assistant greets and asks how they can help
2. Patient describes their issue/reason for visit
3. Patient provides their name
4. Discussion of preferred date and time
5. Patient mentions their doctor's name
6. Confirmation and closing

Uses **Microsoft Phi-4** to generate realistic conversational transcripts.

In [1]:
# === CONFIGURATION ===
NUM_TRANSCRIPTS = 100  # Change this number to generate more or fewer transcripts
OUTPUT_DIR = "/data/generated_transcripts"  # Directory to save the generated transcripts
MODEL_NAME = "microsoft/phi-4"  # Phi-4 model from Hugging Face

# If you have the model downloaded locally, set the path here:
# MODEL_NAME = "/path/to/local/phi-4"

print(f"Configuration:")
print(f"  Number of transcripts to generate: {NUM_TRANSCRIPTS}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Model: {MODEL_NAME}")

Configuration:
  Number of transcripts to generate: 100
  Output directory: /data/generated_transcripts
  Model: microsoft/phi-4


In [2]:
# Install required packages (uncomment if needed)
!pip install transformers accelerate torch

In [8]:
import os
import json
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import random

print("Importing libraries complete.")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Importing libraries complete.
PyTorch version: 2.8.0+cu126
CUDA available: True


In [4]:
# Load Microsoft Phi-4 model
print(f"Loading model: {MODEL_NAME}...")
print("This may take a few minutes on first run.")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# Set padding token if not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model loaded successfully!")
print(f"Device: {next(model.parameters()).device}")

Loading model: microsoft/phi-4...
This may take a few minutes on first run.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Model loaded successfully!
Device: cuda:0


In [5]:
# Prompt templates for generating diverse conversations
PROMPT_TEMPLATE = """Generate a realistic phone conversation transcript between a doctor's office assistant (spk_0) and a patient (spk_1) calling to schedule an appointment.
The conversation that you must generate must include reason for visiting doctors office, patient's name, preferred date/time, and doctor's name. Mark doctors office speaker with spk_0 and patient speaker with spk_1. Make sure that customer doesn't introduce himself first. First introduce yourself, then ask patient about their condition, than their name than doctor and then suggested date and time. Do not suggest doctors name, let patient tell themself who their doctor is. do not suggest time, always ask patient when they have time to visit. Do not check for time, just take suggestion from patient and tell tham that you wrote it down.



Format each line as:
spk_0: [assistant's dialogue]
spk_1: [patient's dialogue]

The conversation should include:
1. Greeting from the assistant
2. Patient describes their medical issue or reason for visit: {issue}
3. Patient provides their full name: {name}
4. Discussion of preferred appointment date and time: {datetime}
5. Patient mentions their doctor: Dr. {doctor}
6. Confirmation and polite closing

Make it natural, focus only on facts, no repeating conditions or names no sympathising with patient

Transcript:
"""

# Sample data for variety
MEDICAL_ISSUES = [
    "persistent headaches for the past week",
    "follow-up for blood pressure monitoring",
    "annual physical examination",
    "persistent cough and congestion",
    "knee pain after exercising",
    "skin rash that won't go away",
    "discuss recent lab results",
    "flu-like symptoms",
    "medication refill and consultation",
    "back pain management",
    "allergic reaction symptoms",
    "routine diabetes check-up",
    "migraine consultation",
    "sprained ankle evaluation",
    "digestive issues"
]

PATIENT_NAMES = [
    "Sarah Johnson",
    "Michael Chen",
    "Emily Rodriguez",
    "David Thompson",
    "Jessica Williams",
    "James Martinez",
    "Amanda Davis",
    "Robert Anderson",
    "Lisa Brown",
    "Christopher Lee",
    "Jennifer Taylor",
    "Daniel Kim",
    "Michelle White",
    "Kevin Patel",
    "Ashley Garcia",
    "Brian Adams",
    "Nicole Miller",
    "Brandon Scott",
    "Stephanie King",
    "Eric Green",
    "Olivia Wilson",
    "Ethan Moore",
    "Sophia Hall",
    "Noah Baker",
    "Ava Wright",
    "Liam Adams",
    "Isabella Carter",
    "Mason Evans",
    "Mia Roberts",
    "Alexander Phillips"
]

APPOINTMENT_TIMES = [
    "tomorrow at 2 PM",
    "next Monday morning around 10 AM",
    "this Friday afternoon, maybe 3 or 4 PM",
    "next week, preferably Wednesday at 11 AM",
    "Thursday at 9:30 AM if possible",
    "early next week, around 8 AM",
    "next Tuesday at 1 PM",
    "sometime this week, afternoon would be best",
    "Monday or Tuesday morning, before 10",
    "end of this week, Friday around noon",
    "next Wednesday at 2:30 PM",
    "early morning on Thursday, 8:30 AM",
    "next Monday afternoon at 4 PM",
    "this Thursday at 10:30 AM",
    "next Friday morning at 9 AM",
    "tomorrow morning at 11 AM",
    "next Tuesday afternoon at 3 PM",
    "this Wednesday morning around 9 AM",
    "next Thursday afternoon, maybe 2 PM",
    "Friday morning at 10 AM if possible",
    "Wednesday afternoon at 1:30 PM",
    "Tuesday morning at 9:00 AM",
    "Thursday evening around 5 PM",
    "Monday morning, any time before noon",
    "Friday afternoon after 1 PM",
    "next week on a Tuesday or Wednesday",
    "sometime next week, late morning",
    "this coming Monday afternoon",
    "any morning this week except Friday",
    "next Friday afternoon, 2 PM"
]

DOCTOR_NAMES = [
    "Smith",
    "Johnson",
    "Williams",
    "Brown",
    "Jones",
    "Garcia",
    "Miller",
    "Davis",
    "Rodriguez",
    "Martinez",
    "Hernandez",
    "Lopez",
    "Gonzalez",
    "Wilson",
    "Anderson"
]

print(f"Loaded {len(MEDICAL_ISSUES)} medical issues")
print(f"Loaded {len(PATIENT_NAMES)} patient names")
print(f"Loaded {len(APPOINTMENT_TIMES)} appointment time options")
print(f"Loaded {len(DOCTOR_NAMES)} doctor names")

Loaded 15 medical issues
Loaded 30 patient names
Loaded 30 appointment time options
Loaded 15 doctor names


In [6]:
def generate_transcript(issue, name, datetime, doctor, max_length=800, temperature=0.8):
    """
    Generate a single transcript of a phone call between doctors office and patient. The conversation should include reason for visiting doctors office, patient's name, preferred date/time, and doctor's name. Mark doctors office speaker with spk_0 and patient speaker with spk_1.

    Args:
        issue: Medical issue or reason for appointment
        name: Patient's name
        datetime: Preferred appointment date/time
        doctor: Doctor's name
        max_length: Maximum tokens to generate
        temperature: Sampling temperature (higher = more creative)

    Returns:
        Generated transcript text
    """
    prompt = PROMPT_TEMPLATE.format(
        issue=issue,
        name=name,
        datetime=datetime,
        doctor=doctor
    )

    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0])

    return generated_text

print("Transcript generation function ready.")

Transcript generation function ready.


In [9]:
issue = random.choice(MEDICAL_ISSUES)
name = random.choice(PATIENT_NAMES)
datetime = random.choice(APPOINTMENT_TIMES)
doctor = random.choice(DOCTOR_NAMES)

transcript = generate_transcript(issue, name, datetime, doctor)

print(transcript)



Generate a realistic phone conversation transcript between a doctor's office assistant (spk_0) and a patient (spk_1) calling to schedule an appointment.
The conversation that you must generate must include reason for visiting doctors office, patient's name, preferred date/time, and doctor's name. Mark doctors office speaker with spk_0 and patient speaker with spk_1. Make sure that customer doesn't introduce himself first. First introduce yourself, then ask patient about their condition, than their name than doctor and then suggested date and time. Do not suggest doctors name, let patient tell themself who their doctor is. do not suggest time, always ask patient when they have time to visit. Do not check for time, just take suggestion from patient and tell tham that you wrote it down.



Format each line as:
spk_0: [assistant's dialogue]
spk_1: [patient's dialogue]

The conversation should include:
1. Greeting from the assistant
2. Patient describes their medical issue or reason for vis

In [10]:
# Create output directory
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)
print(f"Output directory created: {output_path}")

# Generate transcripts
import random

transcripts = []
metadata = []

print(f"\nGenerating {NUM_TRANSCRIPTS} transcripts...\n")

for i in range(NUM_TRANSCRIPTS):
    # Randomly select parameters
    issue = random.choice(MEDICAL_ISSUES)
    name = random.choice(PATIENT_NAMES)
    datetime = random.choice(APPOINTMENT_TIMES)
    doctor = random.choice(DOCTOR_NAMES)

    print(f"[{i+1}/{NUM_TRANSCRIPTS}] Generating transcript for {name}...")
    print(f"  Issue: {issue}")
    print(f"  Doctor: Dr. {doctor}")
    print(f"  Time: {datetime}")

    try:
        transcript = generate_transcript(issue, name, datetime, doctor)

        # Save individual transcript file
        filename = f"transcript_{i+1:03d}.txt"
        filepath = output_path / filename
        filepath.write_text(transcript, encoding="utf-8")

        transcripts.append(transcript)
        metadata.append({
            "id": i + 1,
            "filename": filename,
            "patient_name": name,
            "doctor_name": f"Dr. {doctor}",
            "medical_issue": issue,
            "appointment_time": datetime
        })

        print(f"  ✓ Saved to {filename}")
        print()

    except Exception as e:
        print(f"  ✗ Error generating transcript: {e}")
        print()

print(f"\n{'='*60}")
print(f"Generation complete! Created {len(transcripts)} transcripts.")
print(f"{'='*60}")

Output directory created: /data/generated_transcripts

Generating 100 transcripts...

[1/100] Generating transcript for Kevin Patel...
  Issue: back pain management
  Doctor: Dr. Lopez
  Time: next Tuesday afternoon at 3 PM
  ✓ Saved to transcript_001.txt

[2/100] Generating transcript for Isabella Carter...
  Issue: digestive issues
  Doctor: Dr. Brown
  Time: next Tuesday at 1 PM
  ✓ Saved to transcript_002.txt

[3/100] Generating transcript for Michael Chen...
  Issue: persistent cough and congestion
  Doctor: Dr. Lopez
  Time: this coming Monday afternoon
  ✓ Saved to transcript_003.txt

[4/100] Generating transcript for David Thompson...
  Issue: follow-up for blood pressure monitoring
  Doctor: Dr. Rodriguez
  Time: next Wednesday at 2:30 PM
  ✓ Saved to transcript_004.txt

[5/100] Generating transcript for Daniel Kim...
  Issue: persistent headaches for the past week
  Doctor: Dr. Johnson
  Time: this coming Monday afternoon
  ✓ Saved to transcript_005.txt

[6/100] Generating tr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
# Save metadata as JSON
metadata_file = output_path / "metadata.json"
metadata_file.write_text(json.dumps(metadata, indent=2, ensure_ascii=False), encoding="utf-8")
print(f"Metadata saved to {metadata_file}")

# Display first transcript as example
if transcripts:
    print("\n" + "="*60)
    print("EXAMPLE TRANSCRIPT (First Generated):")
    print("="*60)
    print(transcripts[0])
    print("="*60)

Metadata saved to /data/generated_transcripts/metadata.json

EXAMPLE TRANSCRIPT (First Generated):
Generate a realistic phone conversation transcript between a doctor's office assistant (spk_0) and a patient (spk_1) calling to schedule an appointment.
The conversation that you must generate must include reason for visiting doctors office, patient's name, preferred date/time, and doctor's name. Mark doctors office speaker with spk_0 and patient speaker with spk_1. Make sure that customer doesn't introduce himself first. First introduce yourself, then ask patient about their condition, than their name than doctor and then suggested date and time. Do not suggest doctors name, let patient tell themself who their doctor is. do not suggest time, always ask patient when they have time to visit. Do not check for time, just take suggestion from patient and tell tham that you wrote it down.



Format each line as:
spk_0: [assistant's dialogue]
spk_1: [patient's dialogue]

The conversation should

In [12]:
# Summary statistics
print("\n📊 GENERATION SUMMARY")
print("="*60)
print(f"Total transcripts generated: {len(transcripts)}")
print(f"Output directory: {output_path}")
print(f"\nFiles created:")
for meta in metadata:
    print(f"  - {meta['filename']}: {meta['patient_name']} → Dr. {meta['doctor_name'].replace('Dr. ', '')}")
print(f"  - metadata.json")
print("="*60)
print("\n✅ All transcripts ready for use!")


📊 GENERATION SUMMARY
Total transcripts generated: 100
Output directory: /data/generated_transcripts

Files created:
  - transcript_001.txt: Kevin Patel → Dr. Lopez
  - transcript_002.txt: Isabella Carter → Dr. Brown
  - transcript_003.txt: Michael Chen → Dr. Lopez
  - transcript_004.txt: David Thompson → Dr. Rodriguez
  - transcript_005.txt: Daniel Kim → Dr. Johnson
  - transcript_006.txt: James Martinez → Dr. Jones
  - transcript_007.txt: Michael Chen → Dr. Lopez
  - transcript_008.txt: Ava Wright → Dr. Hernandez
  - transcript_009.txt: Jennifer Taylor → Dr. Brown
  - transcript_010.txt: Daniel Kim → Dr. Rodriguez
  - transcript_011.txt: Robert Anderson → Dr. Brown
  - transcript_012.txt: Stephanie King → Dr. Brown
  - transcript_013.txt: Jessica Williams → Dr. Martinez
  - transcript_014.txt: David Thompson → Dr. Gonzalez
  - transcript_015.txt: Olivia Wilson → Dr. Davis
  - transcript_016.txt: James Martinez → Dr. Davis
  - transcript_017.txt: Michael Chen → Dr. Smith
  - transcrip

In [13]:
!zip -r /data/generated_transcripts.zip /data/generated_transcripts/
from google.colab import files
files.download("/data/generated_transcripts.zip")

  adding: data/generated_transcripts/ (stored 0%)
  adding: data/generated_transcripts/transcript_048.txt (deflated 51%)
  adding: data/generated_transcripts/transcript_060.txt (deflated 55%)
  adding: data/generated_transcripts/transcript_019.txt (deflated 63%)
  adding: data/generated_transcripts/transcript_090.txt (deflated 55%)
  adding: data/generated_transcripts/transcript_007.txt (deflated 55%)
  adding: data/generated_transcripts/transcript_094.txt (deflated 51%)
  adding: data/generated_transcripts/transcript_025.txt (deflated 62%)
  adding: data/generated_transcripts/transcript_050.txt (deflated 58%)
  adding: data/generated_transcripts/transcript_059.txt (deflated 58%)
  adding: data/generated_transcripts/transcript_088.txt (deflated 51%)
  adding: data/generated_transcripts/transcript_029.txt (deflated 51%)
  adding: data/generated_transcripts/transcript_037.txt (deflated 64%)
  adding: data/generated_transcripts/transcript_089.txt (deflated 64%)
  adding: data/generated_tr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Next Steps

The generated transcripts are now saved in the output directory. Each transcript:
- Is saved as a separate `.txt` file
- Follows the format with `spk_0` (assistant) and `spk_1` (patient)
- Contains realistic conversation flow

**To generate more transcripts:**
1. Change the `NUM_TRANSCRIPTS` variable in the configuration cell
2. Re-run the generation cells

**To customize:**
- Add more medical issues, names, or appointment times to the sample data
- Adjust the `temperature` parameter in the generation function for more/less creative outputs
- Modify the prompt template to change conversation structure

**Integration with the rewriter notebook:**
- Point the `ROOT_DIR` in the rewriter notebook to `OUTPUT_DIR` from this notebook
- The rewriter will extract `spk_1` lines and convert them to email format